### Práctica de Acceso a bases de datos

# Considerar la gestión de información sobre coches. Para ello se utilizará una base de datos con el siguiente esquema relacional(en negrita aparecen las claves primarias):

marcas (__cifm__, nombre, ciudad)

coches (__codcoche__, nombre, modelo)

marco (__cifm__, __codcoche__)

concesionarios (__cifc__, nombre, ciudad)

distribucion (__cifc__, __codcoche__, cantidad)

clientes (__dni__, nombre, apellido, ciudad)

ventas (__cifc__, __dni__, __codcoche__, color)


Tabla Marcas:

cifm | Nombre | Ciudad
-- | -- |--
0001| seat|Madrid
0002| renault| Barcelona
0003| citroen| Valencia
0004| audi| Madrid
0005| opel| Bilbao
0006| bmw| Barcelona

Tabla Coches:

codcoche | nombre|modelo
-- | --|--
0001| ibiza|glx
0002| ibiza| gti
0003| ibiza| gtd
0004| toledo| gtd
0005| cordoba| gti
0006| megane| 1.6
0007| megane|gti
0008| laguna| gtd
0009| laguna| td
0010| zx| 16v
0011| zx| td
0012| xantia| gtd
0013| a4 |1.8
0014| a4| 2.8
0015| astra| caravan
0016| astra| gti
0017| corsa| 1.4

Tabla Marco:

cifm | codcoche
-- | --
0001| 0001
0001| 0002
0001| 0003
0001| 0004
0001| 0005
0002| 0006
0002| 0007
0002| 0008
0002| 0009
0003| 0010
0003| 0011
0003| 0012
0004| 0013
0004| 0014
0005| 0015
0005| 0016
0005| 0017

Tabla Concesionarios:

cifc | Nombre | Ciudad
-- | -- |--
0001| acar|Madrid
0002| bcar| Madrid
0003| ccar| Barcelona
0004| dcar| Valencia
0005| ecar| Bilbao

Tabla Distribución

cifc | codcoche| cantidad
-- | --| --
0001| 0001 | 3
0001| 0002 | 7
0001| 0003 | 7
0002| 0006 | 5
0002| 0007 | 10
0002| 0008 | 10
0003| 0010 | 5
0003| 0011 | 3
0003| 0012 | 5
0004| 0013 | 10
0004| 0014 | 5


Tabla Clientes:

dni | nombre | apellidos | ciudad
-- | --|--|--
0001| Luis| Garcia | Madrid
0002| Antonio| López | Valencia
0003| Juan| Martín | Madrid
0004| María| Garcia | Madrid
0005| Javier| González | Barcelona
0006| Ana| López | Barcelona
0007| Ana| López | Madrid
0008| Ana| López | Barcelona


Tabla Ventas:

cifc | dni | codcoche | color
-- | --|--|--
0001| 0001 | 0001 | blanco
0001| 0002 | 0005 | rojo
0002| 0003 | 0008 | blanco
0002| 0001 | 0006 | rojo
0003| 0004 | 0011 | rojo
0004| 0005 | 0014 | verde
0004| 0005 | 0013 | azul
0004| 0004 | 0014 | verde

Esta base de datos contiene información sobre marcas y modelos de coches, los concesionarios que venden los coches, los coches distribuidos a los concesionarios y las ventas realizadas por estos últimos a los clientes.

Se pide:

1.Crear un programa en Python que genere utilizando SQLite una base de datos denominada “Vehículos”  que contenga las tablas mostradas más arriba: Marcas, Coche, Marco, Concesionarios, Distribución, Clientes y Ventas con la información que aparece en cada una de ellas. [2,5 puntos].

2.Crear un programa en Python que resuelva las siguientes consultas SQL [7,5 puntos]. Se debe mostrar por pantalla el resultado de cada consulta:
 
   * Obtener el nombre y el apellido de los clientes que han comprado como mínimo un coche de color ‘verde’ o un coche de color ‘azul’,

   * Obtener el nombre de los concesionarios que sólo han vendido coches al cliente con dni igual a 0001

   * Obtener los codcoche y la media de los valores del atributo cantidad para cada codcoche que aparece en la relación distribucion para los casos en que la media sea menor que 15

   * Obtener para cada valor de dni de los clientes el número de veces que aparece en la relación ventas.

   * Obtener el dni de los clientes de nombre ‘pepe’ o de nombre ‘luis’ (no se admiten valores repetidos)

   * Obtener el cifc de todos los concesionarios cuyo atributo cantidad en la relación distribucion no está comprendido entre 10 y 18 ambos inclusive (es decir, los valores menores que 10 y mayores que 18)

#### Normas de entrega

* Fecha tope de entrega: 17/10/2018
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre BasesDatos_GrupoX donde X será el número de grupo correspondiente.


In [4]:
def crear_conectar_DB(conexion):
    cur=conexion.cursor()
    print('\033[1m',"CREANDO LA BASE DE DATOS...",'\033[0m', "\n")
    cur.execute("DROP TABLE IF EXISTS Marcas")
    cur.execute("CREATE TABLE Marcas(cifm INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, nombre TEXT, ciudad TEXT)")

    cur.execute("DROP TABLE IF EXISTS Coches")
    cur.execute("create table Coches(codigo INTEGER PRIMARY KEY AUTOINCREMENT, nombre TEXT, modelo TEXT)")

    cur.execute("DROP TABLE IF EXISTS Marco")
    cur.execute("create TABLE Marco(cifm NUMBER, codcoche NUMBER, FOREIGN KEY(cifm) REFERENCES Marcas(cifm), FOREIGN KEY(codcoche) REFERENCES Coches(codigo),  PRIMARY KEY(cifm, codcoche))")

    cur.execute("DROP TABLE IF EXISTS Concesionarios")
    cur.execute("create table Concesionarios(cifc INTEGER PRIMARY KEY AUTOINCREMENT, nombre TEXT, ciudad TEXT)")

    cur.execute("DROP TABLE IF EXISTS Distribucion")
    cur.execute("create table Distribucion(cifc  NUMBER, codcoche NUMBER, cantidad NUMBER, FOREIGN KEY(cifc) REFERENCES Concesionarios(cifc), FOREIGN KEY(codcoche) REFERENCES Coches(codigo), PRIMARY KEY(cifc, codcoche))")

    cur.execute("DROP TABLE IF EXISTS Clientes")
    cur.execute("create table Clientes(dni TEXT PRIMARY KEY, nombre TEXT, apellidos TEXT, ciudad TEXT)")

    cur.execute("DROP TABLE IF EXISTS Ventas")
    cur.execute("CREATE TABLE Ventas(cifc  NUMBER, dni TEXT, codcoche NUMBER, color TEXT,  FOREIGN KEY(cifc) REFERENCES Concesionarios(cifm),  FOREIGN KEY(dni) REFERENCES Clientes(dni),  FOREIGN KEY(codcoche) REFERENCES Coches(codigo),  PRIMARY KEY(cifc, dni, codcoche))")
    cur.close()
    conexion.commit()

#Funcion que inserta los datos de datosTabla en la tabla nombreTabla. Los valores insertados están en la variable atributos
def insert(nombreTabla, atributos, datosTabla, conexion):
    cur = conexion.cursor()
    query="INSERT INTO "+nombreTabla + " ("
    parametros = ""
    for atributo in atributos:
        query += atributo + ","
        parametros += "?, "
    query = query.rstrip(",")
    
    parametros = parametros.rstrip(", ")
    query += ") VALUES (" + parametros + ")"
    cur.executemany(query,datosTabla)
    print ("Insertadas", cur.rowcount, "filas en tabla ", nombreTabla)
    cur.close()
    
def insertarDatos(conn):
    camposMarcas = ["nombre", "ciudad"]
    camposCoches = ["nombre", "modelo"]
    camposMarco = ["cifm","codcoche"]
    camposConcesionarios = ["nombre","ciudad"]
    camposDistribucion = ["cifc","codcoche","cantidad"]
    camposClientes = ["dni","nombre","apellidos","ciudad"]
    camposVentas = ["cifc","dni","codcoche","color"]


    marcas = [('seat','Madrid'),('renault', 'Barcelona'),('citroen', 'Valencia'),('audi', 'Madrid'),('opel', 'Bilbao'),('bmw', 'Barcelona')]
    coches = [('ibiza','glx'),('ibiza', 'gti'),('ibiza', 'gtd'),('toledo', 'gtd'),('cordoba', 'gti'),('megane', '1.6'),('megane', 'gti'),('laguna', 'gtd'),('laguna', 'td'),('zx', '16v'),('zx', 'td'),('xantia', 'gtd'),('a4', '1.8'),('a4', '2.8'),('astra', 'caravan'),('astra', 'gti'),('corsa', '1.4')]
    marco = [('1','1'),('1','2'),('1','3'),('1','4'),('1','5'),('2','6'),('2','7'),('2','8'),('2','9'),('3','10'),('3','11'),('3','12'),('4','13'),('4','14'),('5','15'),('5','16'),('5','17')]
    concesionarios = [('acar', 'Madrid'),('bcar', 'Madrid'),('ccar', 'Barcelona'),('dcar', 'Valencia'),('ecar', 'Bilbao')]
    distribucion = [('1', '1', '3'),('1', '2', '7'),('1', '3', '7'),('2', '6', '5'),('2', '7', '10'),('2', '8', '10'),('3', '10', '5'),('3', '11', '3'),('3', '12', '5'),('4', '13', '10'),('4', '14', '5')]
    clientes = [('1', 'Luis', 'Garcia', 'Madrid'),('2', 'Antonio', 'Lopez', 'Valencia'),('3', 'Juan', 'Martin', 'Madrid'),('4', 'Maria', 'Garcia', 'Madrid'),('5', 'Javier', 'Gonzalez', 'Barcelona'),('6', 'Ana', 'Lopez', 'Barcelona'),('7', 'Ana', 'Lopez', 'Madrid'),('8', 'Ana', 'Garcia', 'Barcelona')]
    ventas = [('1', '1', '1', 'blanco'),('1', '2', '5', 'rojo'),('2', '3', '8', 'blanco'),('2', '1', '6', 'rojo'),('3', '4', '11', 'rojo'),('4', '5', '14', 'verde'),('4', '5', '13', 'azul'),('4', '4', '14', 'verde')]

    insert("MARCAS", camposMarcas, marcas, conn)
    insert("COCHES", camposCoches, coches, conn)
    insert("MARCO", camposMarco, marco, conn)
    insert("CONCESIONARIOS", camposConcesionarios, concesionarios, conn)
    insert("DISTRIBUCION", camposDistribucion, distribucion, conn)
    insert("CLIENTES", camposClientes, clientes, conn)
    insert("VENTAS", camposVentas, ventas, conn)
    conn.commit()


def consultas(conexion):
    
    cur=conexion.cursor()
    
    #CONSULTAS--------------------------
    
    print('\033[1m',"\nCONSULTAS:",'\033[0m', "\n")

    #1 - Obtener el nombre y el apellido de los clientes que han comprado como mínimo un coche de color ‘verde’ o un 
    #coche de color ‘azul’,
    cur.execute("SELECT DISTINCT nombre, apellidos FROM Clientes WHERE dni IN (SELECT dni FROM ventas WHERE (color = 'verde' OR color = 'azul'))")
    print('\033[1m',"PRIMERA CONSULTA: Clientes que han comprado un coche verde o rojo",'\033[0m', "\n")
    resultados = cur.fetchall()
    if not resultados:
        print("No hay resultados")
    else:
        x = PrettyTable()
        x.field_names = ["Nombre", "Apellidos"]
        for (nombre, apellidos) in resultados:
                x.add_row([nombre, apellidos])
        print(x, "\n")

    #2 - Obtener el nombre de los concesionarios que sólo han vendido coches al cliente con dni igual a 0001
    cur.execute("SELECT nombre FROM Concesionarios WHERE cifc IN (SELECT DISTINCT cifc FROM Ventas WHERE cifc NOT IN (SELECT DISTINCT cifc FROM Ventas WHERE dni != 1))")
    print ('\033[1m', "SEGUNDA CONSULTA: Nombre de los concesionarios que solo han vendido al cliente con dni = 1",'\033[0m', "\n")
    resultados = cur.fetchall()
    if not resultados:
        print("No hay resultados\n")
    else:
        x = PrettyTable()
        x.field_names = ["Nombre"]
        for (nombre) in resultados:
            nombre = str(nombre).strip("('',)")
            x.add_row([nombre])
        print(x, "\n")


    #3 - Obtener los codcoche y la media de los valores del atributo cantidad para cada codcoche que aparece en la 
    #relación distribucion para los casos en que la media sea menor que 15
    cur.execute("SELECT * FROM (SELECT codcoche, avg(cantidad) as media FROM Distribucion GROUP BY codcoche) WHERE media < 15")

    print ('\033[1m', "TERCERA CONSULTA: Código del coche y la media de los valores del atributo cantidad con media < 15", '\033[0m',"\n")
    resultados = cur.fetchall()
    if not resultados:
        print("No hay resultados\n")
    else:
        x = PrettyTable()
        x.field_names = ["Código coche", "Cantidad"]
        for (codcoche, cantidad) in resultados:
            x.add_row([codcoche, cantidad])
        print(x, "\n")


    #4 - Obtener para cada valor de dni de los clientes el número de veces que aparece en la relación ventas.
    cur.execute("SELECT dni, COUNT(*) FROM ventas GROUP BY dni")
    print ('\033[1m', "CUARTA CONSULTA: Número de veces que cada cliente ha comprado", '\033[0m',"\n")
    resultados = cur.fetchall()
    if not resultados:
        print("No hay resultados\n")
    else:
        x = PrettyTable()
        x.field_names = ["DNI", "Nº compras"]
        for (dni, compras) in resultados:
            x.add_row([dni, compras])
        print(x, "\n")


    #5 - Obtener el dni de los clientes de nombre ‘pepe’ o de nombre ‘luis’ (no se admiten valores repetidos)
    cur.execute("SELECT DISTINCT dni FROM clientes WHERE (nombre = 'Pepe' OR nombre = 'Luis')")
    print ('\033[1m', "QUINTA CONSULTA: DNI de los clientes de nombre Pepe o de nombre Luis",'\033[0m', "\n")
    resultados = cur.fetchall()
    if not resultados:
        print("No hay resultados\n")
    else:
        x = PrettyTable()
        x.field_names = ["DNI"]
        for (dni) in resultados:
            dni = str(dni).strip("('',)")
            x.add_row([dni])
        print(x, "\n")

    #6 - Obtener el cifc de todos los concesionarios cuyo atributo cantidad en la relación distribucion no está comprendido 
    #entre 10 y 18 ambos inclusive (es decir, los valores menores que 10 y mayores que 18)
    cur.execute("SELECT cifc FROM (SELECT cifc, sum(cantidad) as suma FROM Distribucion GROUP BY cifc) WHERE suma <= 10 or suma >= 18") 

    print ('\033[1m', "SEXTA CONSULTA: CIFC de los concesionarios con cantidad en distribucion se encuentra entre 10 y 18", '\033[0m',"\n")
    resultados = cur.fetchall()
    if not resultados:
        print("No hay resultados\n")
    else:
        x = PrettyTable()
        x.field_names = ["CIFC"]
        for (cifc) in resultados:
            cifc = str(cifc).strip("(,)")
            x.add_row([cifc])
        print(x, "\n")
    cur.close()

#Funcion que confirma los cambios y cierra la conexion
def salir(conexion):
    conexion.commit()
    conexion.close()

In [5]:
import sys
#Para poder usar PrettyTable
!{sys.executable} -m pip install PTable

In [6]:
import sys
import sqlite3
from prettytable import PrettyTable
import sqlite3


def ejercicio1(archivoBD):
    conn=sqlite3.connect(archivoBD)
    #Funcion que crea la BBDD
    crear_conectar_DB(conn)
    #Funcion que inserta los datos
    insertarDatos(conn)
    salir(conn)
    
def ejercicio2(archivoBD):
    conn=sqlite3.connect(archivoBD)

    #Funcion para realizar las consultas
    consultas(conn)
    salir(conn)

ejercicio1("vehiculos.sqlite3")
ejercicio2("vehiculos.sqlite3")

 CREANDO LA BASE DE DATOS...  

Insertadas 6 filas en tabla  MARCAS
Insertadas 17 filas en tabla  COCHES
Insertadas 17 filas en tabla  MARCO
Insertadas 5 filas en tabla  CONCESIONARIOS
Insertadas 11 filas en tabla  DISTRIBUCION
Insertadas 8 filas en tabla  CLIENTES
Insertadas 8 filas en tabla  VENTAS
 
CONSULTAS:  

 PRIMERA CONSULTA: Clientes que han comprado un coche verde o rojo  

+--------+-----------+
| Nombre | Apellidos |
+--------+-----------+
| Maria  |   Garcia  |
| Javier |  Gonzalez |
+--------+-----------+ 

 SEGUNDA CONSULTA: Nombre de los concesionarios que solo han vendido al cliente con dni = 1  

No hay resultados

 TERCERA CONSULTA: Código del coche y la media de los valores del atributo cantidad con media < 15  

+--------------+----------+
| Código coche | Cantidad |
+--------------+----------+
|      1       |   3.0    |
|      2       |   7.0    |
|      3       |   7.0    |
|      6       |   5.0    |
|      7       |   10.0   |
|      8       |   10.0   |
|   